In [ ]:
!pip install unidecode
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3
import pandas as pd
import unidecode

## Personalized Ranking

The core use case for personalized ranking is to take a collection of items and to render them in priority or probable order of interest for a user. 
For a VOD application you want dynamically render a personalized shelf/rail/carousel based on some information (director, location, superhero franchise, movie time period etc). This may not be information that you have in your metadata, so a item metadata filter will not work, however you may have this information within you system to generate the item list.

To demonstrate this, we will use the same user from before and a random collection of items.


In [ ]:
%store -r

In [ ]:
items_df = pd.read_csv(dataset_dir + '/movies.csv', sep=',', usecols=[0,1], encoding='UTF-8', dtype={'movieId': "object", 'title': "str"},index_col=0)
rerank_items = items_df.sample(25).index.tolist()

In [ ]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

def get_movie_by_id(movie_id, movie_df=items_df):
    try:
        return movie_df.loc[int(movie_id)]['title']
    except:
        return "Error obtaining title"

In [ ]:
rerank_list = []
for item in rerank_items:
    movie = get_movie_by_id(item)
    rerank_list.append(movie)
rerank_df = pd.DataFrame(rerank_list, columns = ['Un-Ranked'])
rerank_df

In [ ]:
user_id = '593'
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = user_id,
        inputList = rerank_item_list
)

In [ ]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    movie = get_movie_by_id(item['itemId'])
    ranked_list.append(movie)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

In [ ]:
user_id2 = '464'
rerank_item_list2 = []
for item in rerank_items:
    rerank_item_list2.append(str(item))
    
get_recommendations_response_rerank2 = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = user_id2,
        inputList = rerank_item_list
)

rerank_df2 = pd.DataFrame(rerank_list, columns = ['Un-Ranked'])
item_list

In [ ]:
ranked_list2 = []
rerank_df2 = pd.DataFrame(rerank_list, columns = ['Un-Ranked'])
item_list2 = get_recommendations_response_rerank2['personalizedRanking']
for item in item_list2:
    movie2 = get_movie_by_id(item['itemId'])
    ranked_list2.append(movie2)
ranked_df2 = pd.DataFrame(ranked_list2, columns = ['Re-Ranked'])
rerank_df2 = pd.concat([rerank_df2, ranked_df2], axis=1)
rerank_df2

In [ ]:
%store ranked_df2
%store rerank_df2